<a href="https://colab.research.google.com/github/AtishayJain-AJ/DeepWAG/blob/master/Data_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install biopython #Install biopython

In [ ]:
dir = "/content/drive/My Drive/bacteria data/ofloxacin/"  # Directory of the antibiotic

In [ ]:

import os
import numpy as np
from Bio import SeqIO
import pickle as p

In [ ]:
k = 31

kmer_exist_dict = {}

In [ ]:
def kmer_to_number(kmer):
  kmer = kmer.upper()
  base4 = ""
  d = {"A" : "0", "C": "1", "G": "2", "T": "3"}
  if 'N' in kmer:
    return -1
  else:
    for i in kmer:
      base4 = base4 + d[i]

    return int(base4, 4)

In [ ]:
sus_dir = dir + "Susceptible/"
f = os.listdir(sus_dir)
sus_files = []
for i in f:
  if i.endswith("fna"):
    sus_files.append(i)

sus_no = 0

In [ ]:
print(len(f), len(sus_files))

In [ ]:

for file in sus_files:
  sus_no += 1
  file_name = os.path.join(sus_dir, file)
  with open(file_name, "rU") as handle:
    for record in SeqIO.parse(handle, "fasta"):
      seq = record.seq
      for j in range(len(seq) - k + 1):
        kmer_number = kmer_to_number(seq[j:j+k])
        if kmer_number == -1:
          continue
        #kmer_exist[i, kmer_number] = 1
        if kmer_number in kmer_exist_dict:
          if sus_no - 1 not in kmer_exist_dict[kmer_number]:
            kmer_exist_dict[kmer_number].append(sus_no - 1)
        else:
          kmer_exist_dict[kmer_number] = [sus_no - 1]

In [ ]:
res_dir = dir + "Resistant/"
f = os.listdir(res_dir)
res_files = []
for i in f:
  if i.endswith("fna"):
    res_files.append(i)

res_no = 0

In [ ]:
print(len(res_files))

In [ ]:
#resistant_sequences = []

for file in res_files:
  res_no += 1
  file_name = os.path.join(res_dir, file)
  with open(file_name, "rU") as handle:
    for record in SeqIO.parse(handle, "fasta"):
      seq = record.seq
      for j in range(len(seq) - k + 1):
        kmer_number = kmer_to_number(seq[j:j+k])
        if kmer_number == -1:
          continue
        #kmer_exist[i, kmer_number] = 1
        if kmer_number in kmer_exist_dict:
          if sus_no + res_no - 1 not in kmer_exist_dict[kmer_number]:
            kmer_exist_dict[kmer_number].append(sus_no + res_no - 1)
        else:
          kmer_exist_dict[kmer_number] = [sus_no + res_no - 1]
            
       



In [ ]:
kmer_reduced_dict = {}

In [ ]:
seq_no = sus_no + res_no

In [ ]:
threshold = seq_no//100
for i in kmer_exist_dict:
  if threshold < len(kmer_exist_dict[i]) < seq_no - threshold:
    kmer_reduced_dict[i] = list(kmer_exist_dict[i])

In [ ]:
print(len(kmer_reduced_dict))

In [ ]:
equivalent_kmers = []

In [ ]:
reduced_kmers_list = list(kmer_reduced_dict.keys())

In [ ]:
equivalent_kmers.append([reduced_kmers_list[0]])

In [ ]:
for i in reduced_kmers_list[1:]:
  flag = 0
  for j in equivalent_kmers:
    if kmer_reduced_dict[i] == kmer_reduced_dict[j[0]]:
      j.append(i)
      flag = 1
      break
  if flag == 0:
    equivalent_kmers.append([i])

In [ ]:
print(len(equivalent_kmers))

In [ ]:
print(kmer_reduced_dict[equivalent_kmers[100][0]] == kmer_reduced_dict[equivalent_kmers[100][1]])

In [ ]:
final_X = np.zeros((seq_no, len(equivalent_kmers)))

In [ ]:
for i in range(len(equivalent_kmers)):
  for j in kmer_reduced_dict[equivalent_kmers[i][0]]:
    final_X[j][i] = 1

In [ ]:
np.save(dir+'X.npy', final_X)

In [ ]:
y_list = [0]*sus_no + [1]* res_no

In [ ]:
np.save(dir+'Y.npy', np.array(y_list))

In [ ]:
np.save(dir+'equivalent_kmers.npy', np.array(equivalent_kmers))

Testing whether it worked


In [ ]:
X = np.load(dir+'X.npy')

In [ ]:
X.shape

In [ ]:
Y = np.load(dir+'Y.npy')

In [ ]:
Y

In [ ]:
e_kmers = np.load(dir+'equivalent_kmers.npy', allow_pickle = True)

In [ ]:
e_kmers

In [ ]:
len(list(e_kmers))